In [3]:
import cv2
import numpy as np

# Constants for hand gesture detection
THRESHOLD = 60
BLUR = 7
BG_ACCUM_WEIGHT = 0.5

# Initialize background accumulation
accumulate_weight = None

# Start capturing video from default camera
cap = cv2.VideoCapture(0)

# Check if the camera is opened successfully
if not cap.isOpened():
    print("Error opening video capture.")
    exit()

# Function to capture the image
def capture_image():
    ret, frame = cap.read()
    if not ret:
        print("Error capturing frame.")
        return

    # Save the frame as an image
    cv2.imwrite("captured_image.jpg", frame)
    print("Image captured successfully!")

# Function to start video capture
def start_video_capture():
    # Create a VideoWriter object to save the video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter("captured_video.avi", fourcc, 20.0, (640, 480))

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Error capturing frame.")
            break

        # Write the frame to the video file
        out.write(frame)

        # Display the resulting frame
        cv2.imshow("Video Capture", frame)

        # Check for 'q' key press to stop video capture
        if cv2.waitKey(1) == ord('q'):
            break

    # Release the VideoWriter and destroy windows
    out.release()
    cv2.destroyAllWindows()
    print("Video capture stopped.")

# Main loop
while True:
    # Read frame from camera
    ret, frame = cap.read()
    if not ret:
        print("Error capturing frame.")
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (BLUR, BLUR), 0)

    # Accumulate the background
    if accumulate_weight is None:
        accumulate_weight = np.float32(blurred)
    else:
        cv2.accumulateWeighted(blurred, accumulate_weight, BG_ACCUM_WEIGHT)

    # Compute the absolute difference between the current frame and background
    diff = cv2.absdiff(blurred, cv2.convertScaleAbs(accumulate_weight))

    # Threshold the difference image
    _, thresholded = cv2.threshold(diff, THRESHOLD, 255, cv2.THRESH_BINARY)

    # Find contours of hand gesture
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if hand gesture is detected
    if len(contours) > 0:
        # Capture image if the area of the contour is large enough
        if cv2.contourArea(contours[0]) > 500:
            capture_image()

            # Start video capture if the area of the contour is even larger
            if cv2.contourArea(contours[0]) > 2000:
                start_video_capture()

    # Display the frame with contour and threshold images
    cv2.imshow("Hand Gesture", frame)
    cv2.imshow("Thresholded", thresholded)

    # Check for 'q' key press to exit
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture object and destroy windows
cap.release()
cv2.destroyAllWindows()


Image captured successfully!
Image captured successfully!
Image captured successfully!


KeyboardInterrupt: 

In [2]:
import cv2

# Load pre-trained haarcascade for hand detection
hand_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_hand.xml")

# Function to detect hand gestures and capture image
def capture_image_with_index_finger():
    # Open the default camera
    cap = cv2.VideoCapture(0)

    while True:
        # Read the frame from the camera
        ret, frame = cap.read()

        # Convert the frame to grayscale for hand detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect hands in the frame
        hands = hand_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, width, height) in hands:
            # Draw a rectangle around the detected hand
            cv2.rectangle(frame, (x, y), (x+width, y+height), (0, 255, 0), 2)

            # Check if the hand region corresponds to an index finger gesture
            if width * height > 20000:  # Adjust the threshold as needed
                # Save the frame as an image
                cv2.imwrite("captured_image.png", frame)
                print("Image captured!")
                break

        # Display the frame
        cv2.imshow("Capture Image with Index Finger Gesture", frame)

        # Check for 'q' key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Break the loop if an image is captured
        if len(hands) > 0:
            break

    # Release the camera and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function to capture image with index finger gesture
capture_image_with_index_finger()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'
